In [17]:
%reload_ext autoreload

%autoreload 2

import os

import iris
import xarray as xr

# Making new features from existing data

## Precip (pr)

MOOSE data only has lsrain and lssnow (in $kg m^{-2} s^{-1}$ for CPM models. Total precipitation is the sum of these two values.

In [18]:
lssnow_path = f'{os.getenv("DERIVED_DATA")}/moose/london/2.2km-coarsened-27x/rcp85/01/lssnow/day/lssnow_rcp85_land-cpm_london_2.2km-coarsened-27x_01_day_19801201-19811130.nc'
lsrain_path = f'{os.getenv("DERIVED_DATA")}/moose/london/2.2km-coarsened-27x/rcp85/01/lsrain/day/lsrain_rcp85_land-cpm_london_2.2km-coarsened-27x_01_day_19801201-19811130.nc'

# lssnow_ds = xr.open_dataset(lssnow_path)
# lsrain_ds = xr.open_dataset(lsrain_path)
mvar_ds = xr.open_mfdataset([lsrain_path, lssnow_path])


In [19]:
mvar_ds

<xarray.Dataset>
Dimensions:                     (bnds: 2, grid_latitude: 64, grid_longitude: 64, time: 360)
Coordinates:
  * time                        (time) object 1980-12-01 12:00:00 ... 1981-11...
  * grid_latitude               (grid_latitude) float32 -1.58 -1.56 ... -0.32
  * grid_longitude              (grid_longitude) float32 360.9 360.9 ... 362.1
    forecast_period             (time) timedelta64[ns] dask.array<chunksize=(360,), meta=np.ndarray>
    forecast_reference_time     object 1979-09-01 00:00:00
    realization                 int32 1100000
Dimensions without coordinates: bnds
Data variables:
    lsrain                      (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 64, 64), meta=np.ndarray>
    rotated_latitude_longitude  int32 -2147483647
    time_bnds                   (time, bnds) object dask.array<chunksize=(360, 2), meta=np.ndarray>
    grid_latitude_bnds          (grid_latitude, bnds) float32 dask.array<chunksize=(64, 2), meta=np.ndarray>
    grid_longitude_bnds         (grid_longitude, bnds) float32 dask.array<chunksize=(64, 2), meta=np.ndarray>
    forecast_period_bnds        (time, bnds) float64 dask.array<chunksize=(360, 2), meta=np.ndarray>
    lssnow                      (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 64, 64), meta=np.ndarray>
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   10.6
    Conventions:  CF-1.7

In [20]:
pr_ds = mvar_ds.assign(pr=lambda x: x['lssnow']+x['lsrain'])
pr_ds['pr'] = pr_ds['pr'].assign_attrs(units=mvar_ds['lssnow'].attrs['units'], cell_methods=mvar_ds['lssnow'].attrs['cell_methods'], standard_name='pr')
pr_ds = pr_ds.drop_vars(['lssnow', 'lsrain'])
pr_ds

<xarray.Dataset>
Dimensions:                     (bnds: 2, grid_latitude: 64, grid_longitude: 64, time: 360)
Coordinates:
  * time                        (time) object 1980-12-01 12:00:00 ... 1981-11...
  * grid_latitude               (grid_latitude) float32 -1.58 -1.56 ... -0.32
  * grid_longitude              (grid_longitude) float32 360.9 360.9 ... 362.1
    forecast_period             (time) timedelta64[ns] dask.array<chunksize=(360,), meta=np.ndarray>
    forecast_reference_time     object 1979-09-01 00:00:00
    realization                 int32 1100000
Dimensions without coordinates: bnds
Data variables:
    rotated_latitude_longitude  int32 -2147483647
    time_bnds                   (time, bnds) object dask.array<chunksize=(360, 2), meta=np.ndarray>
    grid_latitude_bnds          (grid_latitude, bnds) float32 dask.array<chunksize=(64, 2), meta=np.ndarray>
    grid_longitude_bnds         (grid_longitude, bnds) float32 dask.array<chunksize=(64, 2), meta=np.ndarray>
    forecast_period_bnds        (time, bnds) float64 dask.array<chunksize=(360, 2), meta=np.ndarray>
    pr                          (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 64, 64), meta=np.ndarray>
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   10.6
    Conventions:  CF-1.7

In [22]:
pr_ds.to_netcdf('pr.nc')
iris.load('pr.nc')

[<iris 'Cube' of pr / (kg m-2 s-1) (time: 360; grid_latitude: 64; grid_longitude: 64)>,
<iris 'Cube' of time_bnds / (days since 1980-12-01 00:00:00.000000) (time: 360; -- : 2)>,
<iris 'Cube' of grid_longitude_bnds / (unknown) (grid_longitude: 64; -- : 2)>,
<iris 'Cube' of rotated_latitude_longitude / (unknown) (scalar cube)>,
<iris 'Cube' of grid_latitude_bnds / (unknown) (grid_latitude: 64; -- : 2)>]

In [18]:
mvar_ds.to_netcdf('mvar.nc')
iris.load('mvar.nc')

[<iris 'Cube' of grid_longitude_bnds / (unknown) (grid_longitude: 64; -- : 2)>,
<iris 'Cube' of rotated_latitude_longitude / (unknown) (scalar cube)>,
<iris 'Cube' of grid_latitude_bnds / (unknown) (grid_latitude: 64; -- : 2)>,
<iris 'Cube' of pr / (unknown) (time: 360; grid_latitude: 64; grid_longitude: 64)>,
<iris 'Cube' of time_bnds / (days since 1980-12-01 00:00:00.000000) (time: 360; -- : 2)>,
<iris 'Cube' of stratiform_rainfall_flux / (kg m-2 s-1) (time: 360; grid_latitude: 64; grid_longitude: 64)>,
<iris 'Cube' of stratiform_snowfall_flux / (kg m-2 s-1) (time: 360; grid_latitude: 64; grid_longitude: 64)>]

In [11]:
iris.load(lssnow_path)

[<iris 'Cube' of grid_longitude_bnds / (unknown) (grid_longitude: 64; -- : 2)>,
<iris 'Cube' of rotated_latitude_longitude / (unknown) (scalar cube)>,
<iris 'Cube' of grid_latitude_bnds / (unknown) (grid_latitude: 64; -- : 2)>,
<iris 'Cube' of time_bnds / (hours since 1970-01-01) (time: 360; -- : 2)>,
<iris 'Cube' of stratiform_snowfall_flux / (kg m-2 s-1) (time: 360; grid_latitude: 64; grid_longitude: 64)>]

In [12]:
iris.load(f'{os.getenv("DERIVED_DATA")}/london/2.2km-coarsened-16x/rcp85/01/pr/day/pr_rcp85_land-cpm_uk_2.2km_01_day_19801201-19811130.nc')

[<iris 'Cube' of grid_longitude_bnds / (unknown) (grid_longitude: 53; -- : 2)>,
<iris 'Cube' of rotated_latitude_longitude / (unknown) (scalar cube)>,
<iris 'Cube' of grid_latitude_bnds / (unknown) (grid_latitude: 53; -- : 2)>,
<iris 'Cube' of time_bnds / (hours since 1970-01-01) (time: 360; -- : 2)>,
<iris 'Cube' of lwe_precipitation_rate / (mm/day) (ensemble_member: 1; time: 360; grid_latitude: 53; grid_longitude: 53)>]

In [13]:
iris.load('/home/henry/cdt/summer/data/uk/2.2km/rcp85/01/pr/day/pr_rcp85_land-cpm_uk_2.2km_01_day_19801201-19811130.nc')

Lwe Precipitation Rate (mm/day),ensemble_member,time,grid_latitude,grid_longitude
Shape,1,360,606,484
Dimension coordinates,,,,
ensemble_member,x,-,-,-
time,-,x,-,-
grid_latitude,-,-,x,-
grid_longitude,-,-,-,x
Auxiliary coordinates,,,,
ensemble_member_id,x,-,-,-
month_number,-,x,-,-
year,-,x,-,-
